# Calculation of Metadata Amplification

In [11]:
import pandas as pd
import numpy as np

files = ["./performance-comparison-table-NOVA", "./performance-comparison-table-PMFS"] 
workloads = ["write", "randwrite", "fileserver.f", "varmail.f", "webserver.f", "webproxy.f"]

real_meta_IOs_NOVA = []
real_meta_IOs_PMFS = []
for fid, file in enumerate(files):
    with open(file, "r") as f:
        df = pd.read_csv(f, delim_whitespace=True, engine='python')
    
    breakdown_times = df.columns[14:]
    breakdown_times = breakdown_times.insert(0, "data_read_time(ns)")
    breakdown_times = breakdown_times.insert(1, "data_write_time(ns)")
    
    print("=====================================")
    print(file)
    for workload_idx, workload in enumerate(workloads):
        total_time = 0
        meta_time = 0
        data_time = 0
        print(breakdown_times)
        for idx, breakdown_time in enumerate(breakdown_times):
            total_time += df[df['workloads'] == workload][breakdown_time].iloc[0]
            if idx == 0 or idx == 1:
                data_time += df[df['workloads'] == workload][breakdown_time].iloc[0]
            else:
                meta_time += df[df['workloads'] == workload][breakdown_time].iloc[0]
        # data_time = df[df['workloads'] == workload]["data_time(ns)"].iloc[0]
        # meta_time = df[df['workloads'] == workload]["meta_time(ns)"].iloc[0]
        # total_time = data_time + meta_time

        print("Total IO time of", workload, ": ", total_time / 1000000000)
        print("Meta IO time of", workload, ": ", meta_time / 1000000000)
        print("Data IO time of", workload, ": ", data_time / 1000000000)
        print("Meta IO time percentage of", workload, ": ", meta_time * 100 / total_time, "%")
        print("Data IO time percentage of", workload, ": ", data_time * 100 / total_time, "%")

        for breakdown_time in breakdown_times:
            print(breakdown_time + ": " + str(df[df['workloads'] == workload][breakdown_time].iloc[0] / 1000000000) + "s")
            print(breakdown_time + ": " + str(df[df['workloads'] == workload][breakdown_time].iloc[0] * 100 / total_time) + "%")

        data_write = df[df['workloads'] == workload]["data_write(bytes)"].iloc[0]
        data_read = df[df['workloads'] == workload]["data_read(bytes)"].iloc[0]
        media_write = df[df['workloads'] == workload]["media_write(byte)"].iloc[0]
        media_read = df[df['workloads'] == workload]["media_read(byte)"].iloc[0]
        meta_write = df[df['workloads'] == workload]["meta_write(bytes)"].iloc[0]
        meta_read = df[df['workloads'] == workload]["meta_read(bytes)"].iloc[0]

        software_meta_IO = meta_write + meta_read
        real_meta_IO = 0 if media_write + media_read - media_write - data_write - data_read < 0 else media_write + media_read - media_write - data_write - data_read 
        software_total_IO = software_meta_IO + data_write + data_read

        if fid == 0:
            real_meta_IOs_NOVA.append(real_meta_IO)
        else:
            real_meta_IOs_PMFS.append(real_meta_IO)

        print("workload: ", workload)
        print("software_meta_IO: ", software_meta_IO / 1024 / 1024 / 1024)
        print("real_meta_IO: ", real_meta_IO / 1024 / 1024 / 1024)
        print("software_meta_IO percentage: ", software_meta_IO * 100 / software_total_IO, "%")

    # for workload_idx, workload in enumerate(workloads):


./performance-comparison-table-NOVA
Index(['data_read_time(ns)', 'data_write_time(ns)', 'update_entry_time(ns)',
       'update_inode_time(ns)', 'journal_time(ns)',
       'update_page_tail_time(ns)'],
      dtype='object')
Total IO time of write :  34.658852259
Meta IO time of write :  22.945972416
Data IO time of write :  11.712879843
Meta IO time percentage of write :  66.20522873789488 %
Data IO time percentage of write :  33.79477126210511 %
data_read_time(ns): 0.0s
data_read_time(ns): 0.0%
data_write_time(ns): 11.712879843s
data_write_time(ns): 33.79477126210511%
update_entry_time(ns): 5.821055857s
update_entry_time(ns): 16.795293200998668%
update_inode_time(ns): 2.742980471s
update_inode_time(ns): 7.9142276567676%
journal_time(ns): 9.03e-07s
journal_time(ns): 2.605394988997405e-06%
update_page_tail_time(ns): 14.381935185s
update_page_tail_time(ns): 41.49570527473363%
workload:  write
software_meta_IO:  5.118775148876011
real_meta_IO:  11.26172947883606
software_meta_IO percentag

# KILLER

In [12]:
import pandas as pd
import numpy as np

files = ["./performance-comparison-table-KILLER"] 
workloads = ["write", "randwrite", "fileserver.f", "varmail.f", "webserver.f", "webproxy.f"]
real_meta_IOs_of_KILLER = []

for fid, file in enumerate(files):
    with open(file, "r") as f:
        df = pd.read_csv(f, delim_whitespace=True, engine='python')

    for workload_idx, workload in enumerate(workloads):
        data_write = df[df['workloads'] == workload]["data_write(bytes)"].iloc[0]
        data_read = df[df['workloads'] == workload]["data_read(bytes)"].iloc[0]
        media_write = df[df['workloads'] == workload]["media_write(byte)"].iloc[0]
        media_read = df[df['workloads'] == workload]["media_read(byte)"].iloc[0]
        meta_write = df[df['workloads'] == workload]["meta_write(bytes)"].iloc[0]
        meta_read = df[df['workloads'] == workload]["meta_read(bytes)"].iloc[0]

        data_time = df[df['workloads'] == workload]["data_time(ns)"].iloc[0]
        meta_time = df[df['workloads'] == workload]["meta_time(ns)"].iloc[0]
        total_time = data_time + meta_time

        meta_time = df[df['workloads'] == workload]["meta_time(ns)"].iloc[0]
        update_bm_time = df[df['workloads'] == workload]["update_bm_time(ns)"].iloc[0]
        update_package_time = df[df['workloads'] == workload]["update_package_time(ns)"].iloc[0]

        software_meta_IO = meta_write + meta_read
        real_meta_IO = 0 if media_write + media_read - media_write - data_write - data_read < 0 else media_write + media_read - media_write - data_write - data_read 
        software_total_IO = software_meta_IO + data_write + data_read
        
        real_meta_IOs_of_KILLER.append(real_meta_IO)

        print("workload: ", workload)
        print("software_meta_IO: ", software_meta_IO / 1024 / 1024 / 1024)
        print("real_meta_IO: ", real_meta_IO / 1024 / 1024 / 1024)
        print("software_meta_IO percentage: ", software_meta_IO * 100 / software_total_IO, "%")
        print("Total IO time (seconds) of", workload, ": ", total_time / 1000000000)
        print("meta percentage: ", meta_time * 100 / total_time, "%")
        print("meta time: ", meta_time / 1000000000, "s")
        print("bm percentage: ", update_bm_time * 100 / total_time, "%")
        print("bm time: ", update_bm_time / 1000000000, "s")
        print("package percentage: ", update_package_time * 100 / total_time, "%")
        print("package time: ", update_package_time / 1000000000, "s")
        print("")
        
        # software_meta_IO = meta_write + meta_read
        # real_meta_IO = 0 if media_write + media_read - media_write - data_write - data_read < 0 else media_write + media_read - media_write - data_write - data_read 

        # print("workload: ", workload)
        # print("software_meta_IO: ", software_meta_IO / 1024 / 1024 / 1024)
        # print("real_meta_IO: ", real_meta_IO / 1024 / 1024 / 1024)


workload:  write
software_meta_IO:  0.06335531454533339
real_meta_IO:  0.8576929569244385
software_meta_IO percentage:  0.19759415046806614 %
Total IO time (seconds) of write :  11.010048628
meta percentage:  37.120894594472084 %
meta time:  4.087028546 s
bm percentage:  0.0016223724893070473 %
bm time:  0.000178624 s
package percentage:  37.119272221982776 %
package time:  4.086849922 s

workload: 

 randwrite
software_meta_IO:  0.5001223925501108
real_meta_IO:  2.970576286315918
software_meta_IO percentage:  1.5388323357968403 %
Total IO time (seconds) of randwrite :  16.776661776
meta percentage:  68.9908622557904 %
meta time:  11.574363617 s
bm percentage:  0.22853092892918317 %
bm time:  0.038339861 s
package percentage:  68.76233132686122 %
package time:  11.536023756 s

workload:  fileserver.f
software_meta_IO:  0.2887579519301653
real_meta_IO:  3.9978789500892162
software_meta_IO percentage:  0.2831384969084034 %
Total IO time (seconds) of fileserver.f :  39.409715277
meta percentage:  20.7223540555903 %
meta time:  8.166620732 s
bm percentage:  0.09320718696345835 %
bm time:  0.036732687 s
package percentage:  20.629146868626844 %
package time:  8.129888045 s

workload:  varmail.f
software_meta_IO:  0.7148057129234076
real_meta_IO:  13.26187111157924
software_meta_IO percentage:  3.8360211409856855 %
Total IO time (seconds) of varmail.f :  22.633223522
meta percentage:  58

# Reduced Metadata I/Os 

In [15]:
min_amps = []
max_amps = []
for idx, meta_IO in enumerate(real_meta_IOs_of_KILLER):
    if meta_IO == 0:
        continue
    amp_NOVA = real_meta_IOs_NOVA[idx] / meta_IO 
    amp_PMFS = real_meta_IOs_PMFS[idx] / meta_IO
    min_amp = min(amp_NOVA, amp_PMFS)
    min_amps.append(min_amp)
    max_amp = max(amp_NOVA, amp_PMFS)
    max_amps.append(max_amp)
    
print("min_amps: ", min_amps)
print("max_amps: ", max_amps)

min_amps:  [10.762254668269666, 4.216790962719211, 2.0504739960905014, 1.2680023528071733, 1.192159030959834]
max_amps:  [13.130257614821712, 4.547970945864601, 3.2850315619681774, 1.6535455038632634, 1.3144312432540042]
